In [3]:
!kubectl -n argocd get all 

NAME                                                    READY   STATUS    RESTARTS       AGE
pod/argocd-application-controller-0                     1/1     Running   17 (41h ago)   6d21h
pod/argocd-applicationset-controller-58965fcf4c-6sld4   1/1     Running   12 (41h ago)   5d
pod/argocd-dex-server-5bd46459b7-wjpv7                  1/1     Running   16 (41h ago)   7d22h
pod/argocd-notifications-controller-77fb46fc9-tpjqx     1/1     Running   6 (41h ago)    3d14h
pod/argocd-redis-87c5b59cf-q72xp                        1/1     Running   18 (41h ago)   7d22h
pod/argocd-repo-server-7b589485ff-p8p7k                 1/1     Running   9 (41h ago)    5d
pod/argocd-server-5d78bcbc7b-xd7vg                      1/1     Running   18 (41h ago)   6d21h

NAME                                       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)             AGE
service/argocd-applicationset-controller   ClusterIP   10.110.159.50   <none>        7000/TCP            8d
service/argocd-dex-server     

# Ingress

In [2]:
!kubectl -n argocd get ingress argocd-server -o yaml | yq

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    meta.helm.sh/release-name: argocd
    meta.helm.sh/release-namespace: argocd
  creationTimestamp: "2024-01-29T09:41:25Z"
  generation: 1
  labels:
    app.kubernetes.io/component: server
    app.kubernetes.io/instance: argocd
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: argocd-server
    app.kubernetes.io/part-of: argocd
    app.kubernetes.io/version: v2.9.5
    helm.sh/chart: argo-cd-5.53.3
  name: argocd-server
  namespace: argocd
  resourceVersion: "671333"
  uid: 17a45eec-3882-4d54-a7c9-e78f3821336a
spec:
  ingressClassName: nginx
  rules:
    - host: argocd.worldl.xpt
      http:
        paths:
          - backend:
              service:
                name: argocd-server
                port:
                  number: 80
            path: /
            pathType: Prefix
status:
  loadBalancer:
    ingress:
      - ip: 192.168.49.2
